# EU Energy Map

**Version 1.1**

https://github.com/kuranez/EU-Energy-Map

## **Code Structure**

### I. Import
a. Standard Library Modules (Built-in Python modules)
b. Interactive Dashboard and Web App Development
c. Data Manipulation and Geospatial Analysis
d. Data Visualization (Plotly Ecosystem)

#### **Code**

In [1]:
## a. Standard Library Modules (Built-in Python modules)
import os
import json

## b. Interactive Dashboard and Web App Development
import panel as pn

## c. Data Manipulation and Geospatial Analysis
import pandas as pd
import geopandas as gpd

## d. Data Visualization (Plotly Ecosystem)
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
from plotly.colors import sample_colorscale

### II. Settings
a. Set Panel extension
b. Set Plotly renderer
c. Set MapBox-Access-Token

#### **Code**

In [ ]:
## a. Set up Panel extension
pn.extension('tabulator', 'plotly', design='material', sizing_mode='stretch_width')  

## b. Set Plotly renderer to JupyterLab
# pio.renderers.default = "jupyterlab"

## c. Set MapBox-Access-Token
mapbox_token = 'pk.eyJ1Ijoia3VyYW5leiIsImEiOiJjbTJmMjI0d2kwNDVxMnFzYXNldnc1N2VsIn0.t11TYpF2QBdid-hQfW8mig'

### III. Data Processing

#### a. Load data
- **Step 1:** Load CSV data
- **Step 2:** Load GeoJSON as dict
- **Step 3:** Load GeoJSON into GeoDataFrame

#### **Code**

In [ ]:
## Step 1: Load CSV data
data = pd.read_csv('./data/nrg_ind_ren_linear.csv')  

## Step 2: Load GeoJSON for European countries
with open('europe.geojson') as f:
europe_map = json.load(f)

## Step 3: Load the GeoJSON into a GeoDataFrame
europe_gdf = gpd.read_file("europe.geojson")

#### b. Merge data

- **Step 4:** Merge energy data with map data

#### **Code**

In [ ]:
## Step 4: Merge energy data with map data
merged_data = gpd.read_file('europe.geojson').merge(data, left_on='CNTR_ID', right_on='geo')

#### c. Format data

- **Step 5:** Clean and rename columns
- **Step 6:** Rename entries
- **Step 7:** Drop columns
- **Step 8:** Convert to numeric
- **Step 9:** Round numbers
- **Step 10:** Add country flags
- **Step 11:** Reorder columns
- **Step 12:** Apply columns order

#### **Code**

In [ ]:
## Step 5: Clean and rename columns for readability
merged_data = merged_data.rename(columns={
	'nrg_bal': 'Energy Type',
	'TIME_PERIOD': 'Year',
	'OBS_VALUE': 'Renewable Percentage',
	'geo': 'Code',
	'NAME_ENGL': 'Country',
})

## Step 6: Rename entries in 'Energy Type' column for clarity
merged_data['Energy Type'] = merged_data['Energy Type'].replace({
	'REN': 'Renewable Energy Total',
	'REN_ELC': 'Renewable Electricity',
	'REN_HEAT_CL': 'Renewable Heating and Cooling',
	'REN_TRA': 'Renewable Energy in Transport'
})

## Step 7: Drop columns
merged_data = merged_data.drop(columns=['LAST UPDATE', 'freq', 'unit', 'OBS_FLAG'])

## Step 8: Convert to numeric
merged_data[['Year', 'Renewable Percentage']] = merged_data[['Year', 'Renewable Percentage']].apply(pd.to_numeric)

## Step 9: Round the 'Renewable Percentage' to 1 decimal place
merged_data['Renewable Percentage'] = merged_data['Renewable Percentage'].round(1)

## Step 10: Add Country Flags

@pn.cache
# Function to convert ISO2 country code to flag emoji
def iso2_to_flag(iso2_code):
# Convert ISO2 code to country flag emoji using the formula
return chr(0x1F1E6 + ord(iso2_code[0]) - ord('A')) + chr(0x1F1E6 + ord(iso2_code[1]) - ord('A'))

# Add a new column 'Flag' with the corresponding country flag emojis
merged_data['Flag'] = merged_data['Code'].apply(iso2_to_flag)

## Step 11: Reorder columns
final_columns_order = [
	'Code', 'Flag', 'Country', 'Energy Type', 'Renewable Percentage', 'Year',
	'SVRG_UN', 'EU_STAT', 'EFTA_STAT', 'CC_STAT',
	'CNTR_ID', 'CNTR_NAME', 'NAME_FREN', 'NAME_GERM',
	'ISO3_CODE',
	'CAPT',
	'DATAFLOW',
	'geometry',
]

## Step 12: FINAL DATAFRAME: Apply column order
merged_data_clean = merged_data[final_columns_order]

#### d. Filter data

- **Step 1:** Filter by EU countries
- **Step 2:** Filter by Energy type
- **Step 3:** Filter by Year
- **Step 4:** Filter by Countriy

#### **Code**

In [ ]:
## Step 1: Filter by EU countries
eu_countries = [
	"AT", "BE", "BG", "HR", "CY", "CZ", "DK", "EE", "FI", "FR", "DE", "GR", "HU", "IE", "IT", "LV", "LT", "LU", "MT", "NL", "PL", "PT", "RO", "SK", "SI", "ES", "SE"

]

eu_data = merged_data_clean[merged_data_clean['Code'].isin(eu_countries)]

## Step 2: Filter by Energy type
df_renewable = merged_data_clean[
(merged_data_clean['Energy Type'] == 'Renewable Energy Total') &
(merged_data_clean['Code'].isin(eu_countries))]

## Step 3: Filter by Year
selected_year = 2022 # for testing/initial display
df_year = df_renewable[df_renewable['Year'] == selected_year]

## Step 4: Filter by Country
selected_country = "Germany" # for testing/iitial display
df_country = df_renewable[df_renewable['Country'] == selected_country]


#### e.  Data Calculations

- EU Total Average Renewable Energy Percentage

#### **Code**

In [ ]:
# EU Total Average
# - Group by year and calculate the average renewable energy percentage for the EU total
df_eu_total = df_renewable.groupby('Year', as_index=False)['Renewable Percentage'].mean()

### IV. Dashboard Components

#### a. Widgets

- (1) **Year Selection** (year_slider)
- (2) **Country Selection** (country_selection)

#### **Code**

In [2]:
# (1) YEAR SELECTION
year_slider = pn.widgets.IntSlider(name='Select Year', start=2004, end=2022, step=1, value=2022)

# Extract unique country names
unique_countries = df_renewable['Country'].unique().tolist()
unique_flags = df_renewable['Flag'].unique().tolist()

# Combine country names and flags
country_flag_items = [f"{flag} {country}" for country, flag in zip(unique_countries, unique_flags)]

# Define the default country
default_country = "Germany"

# Find the corresponding entry in `country_flag_items`
default_value = next(item for item in country_flag_items if default_country in item)

# (2) COUNTRY SELECTION
country_selection = pn.widgets.Select(name='Select country', options=country_flag_items, value=default_value)

NameError: name 'df_renewable' is not defined

#### b. Sidebar

- str: sidebar_content
- (3) sidebar_pane

#### **Code**

In [3]:
sidebar_content = """
#### About

Explore renewable energy developments in the European Union. Data is sourced from [Eurostat](https://ec.europa.eu/eurostat/databrowser/explore/all/envir?lang=en&subtheme=nrg&display=list&sort=category).

#### Project Page on GitHub

[https://github.com/kuranez/EU-Energy-Map](https://github.com/kuranez/EU-Energy-Map) \

"""

# (3) SIDEBAR PANE
sidebar_pane = pn.pane.Markdown(sidebar_content)


#### c. Main Panel Components

- (4) **Table** (fig_table)
- (5) **Map** (create_choropleth_map(df_year), returns fig_map)
- (6) **Year Chart** (create_bar_chart_year(df_year, year), returns fig)
- (7) **Country Chart** (create_bar_chart_country(df_eu_total, df_country, country), returns fig)

#### **Code**

In [4]:
# (4) Display data in TABULATOR WIDGET
fig_table = pn.widgets.Tabulator(
	df_renewable[['Country', 'Code', 'Flag', 'Energy Type', 'Renewable Percentage', 'Year']],
	pagination="remote", # Enables pagination
	page_size=10, # Sets the number of rows per page
	show_index=False # Hides the index column
)

NameError: name 'df_renewable' is not defined

In [5]:
@pn.cache
#(5) Create Choropleth map function
def create_choropleth_map(df_year): 
	fig_map = go.Figure(go.Choroplethmapbox(
		geojson=europe_map,
		locations=df_year['Code'],
		z=df_year['Renewable Percentage'],
		colorscale="Viridis",
		zmin=0, zmax=100,
		marker_opacity=0.8, marker_line_width=0.5,
		featureidkey="properties.CNTR_ID",
		hoverinfo='location+z', hovertext=df_year['Renewable Percentage']
	))

	fig_map.update_layout(
		mapbox_accesstoken=mapbox_token,
		mapbox_style="carto-positron",
		mapbox_zoom=3,
		mapbox_center={"lat": 54, "lon": 15},
		title="Renewable Energy as Percentage by Country in 2022",
		margin={"r": 0, "t": 0, "l": 0, "b": 0},
		geo=dict(scope='europe', showlakes=False)
	)

	return fig_map


In [6]:
# (6) Create bar chart by year for all countries (create_bar_chart_year)
@pn.cache
def create_bar_chart_year(df_year, year):
	# Sort dataframe by renewable percentage in ascending order
	df_year = df_year.sort_values(by='Renewable Percentage')

	# Calculate EU total average renewable percentage for 2022
	eu_total_avg = df_year['Renewable Percentage'].mean()

	# Map EU total average to a color in the Viridis scale
	color_scale = px.colors.sequential.Viridis
	normalized_avg = eu_total_avg / 100 # Normalize average to a 0-1 range
	scaled_color = sample_colorscale(color_scale, normalized_avg)[0]

	# Create bar trace (main layer)
	bar_trace = go.Bar(
		x=df_year['Country'],
		y=df_year['Renewable Percentage'],
		marker=dict(
			color=df_year['Renewable Percentage'],
			coloraxis='coloraxis', # Link to coloraxis
		),
		name="", # Fix to hide trace info
		hovertemplate="Country: %{customdata[0]}"
					"<b>%{customdata[1]}</b><br>"
					"Renewable Percentage: <b>%{y:.1f}%</b>",
		customdata=df_year[['Flag', 'Country']].values, # Attach custom data
		showlegend=False,
	)

	# Add scatter trace for EU Total Average (border layer)
	scatter_trace_border = go.Scatter(
		x=df_year['Country'], # Use all countries to make the line span across
		y=[eu_total_avg] * len(df_year), # Duplicate average value for each country
		mode="lines",
		line=dict(dash="solid", color="rgba(255, 255, 255, 0.7)", width=4),
		hoverinfo="skip", # Suppress hover for border layer
		showlegend=False,
	)

	# Add scatter trace for EU Total Average (main layer)
	scatter_trace = go.Scatter(
		x=df_year['Country'], # Use all countries to make the line span across
		y=[eu_total_avg] * len(df_year), # Duplicate average value for each country
		mode="lines",
		line=dict(dash="solid", color=scaled_color, width=2),
		hovertemplate="🇪🇺 EU Total Average:<b> %{y:.1f}%</b>",
		name="", # Fix to suppress showing trace info
		showlegend=False,
	)

	# Create figure
	fig = go.Figure(data=[bar_trace, scatter_trace_border, scatter_trace])
	
	# Update layout
	fig.update_layout(
		title=f"Renewable Energy Percentage by Country in {year}",
		xaxis=dict(title=None),
		yaxis=dict(title="Renewable Energy (%)"),
		coloraxis=dict( # Define coloraxis for color bar
			colorscale='Viridis',
			cmin=0,
			cmax=100,
			colorbar=dict(
				orientation="v",
				title=None,
				tickvals=[0, 20, 40, 60, 80, 100],
				ticktext=["0%", "20%", "40%", "60%", "80%", "100%"],
			),
		),
		margin={"t": 50, "b": 50, "l": 50, "r": 50},
		height=450,
	)

return fig

SyntaxError: 'return' outside function (719737881.py, line 75)

In [7]:
@pn.cache
def create_bar_chart_country(df_eu_total, df_country, country):
	# Calculate the average renewable energy percentage for all years (used for annotation)
	eu_total_avg = df_eu_total['Renewable Percentage'].mean()

	# Map EU total average to a color in the Viridis scale
	color_scale = px.colors.sequential.Viridis
	normalized_avg = eu_total_avg / 100 # Normalize average to a 0-1 range
	scaled_color = sample_colorscale(color_scale, normalized_avg)[0]
	
	# Create the figure with Plotly
	fig = go.Figure()
	
	# Add the line trace for EU total renewable energy percentage over the years
	fig.add_trace(go.Scatter(
		x=df_eu_total['Year'],
		y=df_eu_total['Renewable Percentage'],
		mode='lines+markers',
		line=dict(color=scaled_color, width=3),
		marker=dict(
			size=6,
			color=df_eu_total['Renewable Percentage'], # Map the renewable percentage to color
			colorscale='Viridis',
			coloraxis="coloraxis",
		),
		hovertemplate="🇪🇺 %{x}: <b>%{y:.1f}%</b>",
		name="EU Total",
	))

	# Add the bar trace for the selected country's renewable energy percentage
	fig.add_trace(go.Bar(
		x=df_country['Year'],
		y=df_country['Renewable Percentage'],
		marker=dict(
			color=df_country['Renewable Percentage'], # Map the renewable percentage to color
			colorscale='Viridis',
			coloraxis="coloraxis",
		),
		customdata=df_country[['Flag', 'Country']].values, # Attach custom data
		hovertemplate="Country: %{customdata[0]} "
						"<b>%{customdata[1]}</b><br>"
						"Year: <b>%{x}</b><br>"
						"Renewable Percentage: <b>%{y:.1f}%</b>",
		name="",
		showlegend=False,
	))

	# Define the layout with legend at the bottom
	fig.update_layout(
		title=f"Renewable Energy Percentage ({country}, 2004-2022)",
		xaxis=dict(
			title="Year",
			showgrid=False,
			showline=True,
			tickmode="linear",
			tick0=2005,
			dtick=5,
			range=[2003.5, 2022.5],
		),
		yaxis=dict(
			title="Renewable Energy (%)",
		),
		coloraxis=dict(
			colorscale='Viridis',
			cmin=0,
			cmax=100,
			colorbar=dict(
				orientation="v",
				tickvals=[0, 20, 40, 60, 80, 100],
				ticktext=["0%", "20%", "40%", "60%", "80%", "100%"],
			),
		),
		height=450,
		margin={"t": 50, "b": 50, "l": 50, "r": 50},
		showlegend=True,
		legend=dict(
			orientation="h",
			yanchor="top",
			y=1-0.01,
			xanchor="left",
			x=0+0.01
		),
	)

return fig

SyntaxError: 'return' outside function (3733556920.py, line 85)

### V. Dashboard Creation

#### a. Create Components

- Step 1: Create map (eu_map)
- Step 2: Create year chart (fig_by_year)
- Step 3: Create country chart (fig_by_country)

#### **Code**

In [8]:
## Step 1: Create map (eu_map)
eu_map = create_choropleth_map(df_year)

## Step 2: Create year chart (fig_by_year)
year = selected_year
fig_by_year = create_bar_chart_year(df_year, year)
  
## Step 3: Create country chart (fig_by_country)
country = selected_country
fig_by_country = create_bar_chart_country(df_eu_total, df_country, country)

NameError: name 'df_year' is not defined

#### b. Update Functions

- 1. Update map by year 
- 2. Update year chart 
- 3. Update country chart by country
- 4. Update country chart by selection

#### **Code**

##### a. Update map by year (update_map)

In [9]:
@pn.cache
def update_map(year):
	df_selected_year = df_renewable[df_renewable['Year'] == year]
	return create_choropleth_map(df_selected_year)

##### b. Update year chart (update_year_chart)

In [13]:
@pn.cache
def update_year_chart(year):
	df_selected_year = df_renewable[df_renewable['Year'] == year].sort_values(by='Renewable Percentage')
	return create_bar_chart_year(df_selected_year, year)

##### c. Update country chart (update_country_chart)

In [11]:
@pn.cache
def update_country_chart(country):
	df_selected_country = df_renewable[df_renewable['Country'] == country]
	return create_bar_chart_country(df_eu_total, df_selected_country, country)

##### d. Update country chart by selection (update_country_chart)

In [12]:
def update_country_chart(selected_value):
	# Extract the country name from the selection
	selected_country = selected_value.split(" ", 1)[1]
	# Filter the dataframe for the selected country
	df_selected_country = df_renewable[df_renewable['Country'] == selected_country]
	# Create the bar chart
	return create_bar_chart_country(df_eu_total, df_selected_country, selected_country)

#### c. Bind Widgets to Update Functions

- Step 1: Bind year selection to map 
- Step 2: Bind year yelection to chart
- Step 2: Bind country selection

#### **Code**

In [ ]:
## Step 1: Bind year selection to map
interactive_map = pn.bind(update_map, year_slider)

## Step 2: Bind year selection to chart
interactive_bar_chart_year = pn.bind(update_year_chart, year_slider)

## Step 3: Bind country selection to chart
interactive_bar_chart_country = pn.bind(update_country_chart, country_selection.param.value)

### VI. Dashboard Layout

#### a. Create Panes

- i. Year selection & chart (filter_year_pane)
- ii. Country selection & chart (filter_country_pane)
- iii. Data table (table_pane)

#### **Code**

In [14]:
## i. Year selection & chart
filter_year_pane = pn.Column(
	year_slider,
	pn.pane.Plotly(interactive_bar_chart_year, sizing_mode='stretch_width')
)

## ii. Country selection & chart
filter_country_pane = pn.Column(
	country_selection,
	pn.pane.Plotly(interactive_bar_chart_country, sizing_mode='stretch_width')
)

## iii. Data table
table_pane = pn.Column(fig_table)


NameError: name 'interactive_bar_chart_year' is not defined

#### b. Create Tabs (tabs)

#### **Code**

In [15]:
tabs = pn.Tabs(
	('Filter by Year', filter_year_pane),
	('Filter by Country', filter_country_pane)
	('Explore Data', table_pane)
)

<>:3: SyntaxWarning: 'tuple' object is not callable; perhaps you missed a comma?
<>:3: SyntaxWarning: 'tuple' object is not callable; perhaps you missed a comma?
/tmp/ipykernel_41698/1174661310.py:3: SyntaxWarning: 'tuple' object is not callable; perhaps you missed a comma?
  ('Filter by Country', filter_country_pane)


NameError: name 'filter_year_pane' is not defined

#### c. Create Layout (layout)

#### **Code**

In [16]:
layout = pn.Column(
	pn.pane.Plotly(interactive_map, sizing_mode='stretch_width'),
	tabs,
	sizing_mode='stretch_width'
)

NameError: name 'interactive_map' is not defined

### VII. Serve Dashboard

#### **Code**

In [ ]:
pn.template.FastListTemplate(
	title="EU Energy Map", 
	sidebar=[sidebar_pane],
	main=[layout]
).servable();